In [16]:
import asyncio
import load_env
import time
import src.graphql.models.recipe_models_autogen as rma
from sqlalchemy import select, and_
from src.graphql.db.session import get_session
from src.graphql.scalars.recipe_scalar import NEI_Item, NEI_Fluid, NEI_Recipe_Dimensions, NEI_GT_Recipe
from src.graphql.resolvers.recipe_resolver import _getNEIRecipe, _getNEIGTRecipe

In [17]:
# Test joins
recipe_ids = ['r~--eEYPHsN5GqWygrEHan4A==']
item_meta = 'i~gregtech~gt.metaitem.01~23019'

combined_stmt = select(rma.RecipeType, rma.GregTechRecipe) \
    .join(rma.Recipe, rma.RecipeType.id == rma.Recipe.recipe_type_id) \
    .join(rma.GregTechRecipe, rma.GregTechRecipe.recipe_id == rma.Recipe.id) \
    .filter(and_(rma.Recipe.id == recipe_ids[0]))

# print(recipe_type_stmt)
# print(gt_info_stmt)
print(combined_stmt)

SELECT recipe_type.id, recipe_type.category, recipe_type.fluid_input_dimension_height, recipe_type.fluid_input_dimension_width, recipe_type.fluid_output_dimension_height, recipe_type.fluid_output_dimension_width, recipe_type.icon_info, recipe_type.item_input_dimension_height, recipe_type.item_input_dimension_width, recipe_type.item_output_dimension_height, recipe_type.item_output_dimension_width, recipe_type.shapeless, recipe_type.type, recipe_type.icon_id, greg_tech_recipe.id AS id_1, greg_tech_recipe.additional_info, greg_tech_recipe.amperage, greg_tech_recipe.duration, greg_tech_recipe.requires_cleanroom, greg_tech_recipe.requires_low_gravity, greg_tech_recipe.voltage, greg_tech_recipe.voltage_tier, greg_tech_recipe.recipe_id 
FROM recipe_type JOIN recipe ON recipe_type.id = recipe.recipe_type_id JOIN greg_tech_recipe ON greg_tech_recipe.recipe_id = recipe.id 
WHERE recipe.id = :id_2


In [18]:
def _prepORMDict(item, rename={}, exclude=[], include={}):
    d = dict(item.__dict__)
    d.pop('_sa_instance_state')

    for k, v in rename.items():
        d[v] = d.pop(k)
    for k in exclude:
        d.pop(k)
    d.update(include)
    
    return d


async with get_session() as session:
    # awaitables = [session.execute(gt_info_stmt), session.execute(recipe_type_stmt)]
    # gtRecipeORM, recipeTypeORM = [x.scalar_one() for x in await asyncio.gather(*awaitables)]
    start = time.time()
    result = await session.execute(combined_stmt)
    recipeTypeORM, gtRecipeORM = result.one()
    print(f"Query took {time.time() - start} seconds")

    print(gtRecipeORM.__dict__)


Query took 0.010071992874145508 seconds
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f388b828790>, 'requires_cleanroom': False, 'additional_info': '', 'amperage': 1, 'voltage': 30, 'recipe_id': 'r~--eEYPHsN5GqWygrEHan4A==', 'id': 'gtr~r~--eEYPHsN5GqWygrEHan4A==', 'duration': 20, 'requires_low_gravity': False, 'voltage_tier': 'LV'}
